In [ ]:
from pathlib import Path
import altair as alt 
import pandas as pd 
from sklearn.decomposition import NMF
alt.data_transformers.disable_max_rows()
%load_ext autoreload
%autoreload 2

In [ ]:
def big_chart(chart, fontsize = 20): 
    return chart.configure_axis(
    labelFontSize = fontsize,
    titleFontSize = fontsize
).configure_title(
    fontSize = fontsize
    )

In [ ]:
data_df = pd.read_pickle(Path().absolute().parent / 'data'/'READING_2016_preprocessed.pkl')

In [ ]:
data_df = (
    data_df.Consumption
    .unstack().rename_axis(columns = 'timestamp')
)

In [ ]:

data_df

In [ ]:
# IDX = 10
IDX = 15
profile_vis = data_df.iloc[IDX].to_frame("value").rename_axis(index = 'time').reset_index()
profile_vis

In [ ]:
ts_chart = alt.Chart(profile_vis, width = 1200, height = 200).mark_line().encode(
    x = alt.X('time:T', title = None, axis = alt.Axis( format = '%b', values = list(pd.date_range('2016-1-1', '2016-12-31',periods = 12))), ), 
    y = alt.Y('value:Q', title = 'Consumption')
)

big_chart(ts_chart)

In [ ]:
vis_df = profile_vis[(profile_vis.time.dt.isocalendar().week == 17)].time.agg(['min','max']).to_frame('time').T
# rect_chart = alt.Chart(vis_df).mark_rect(color='black', strokeWidth = 2, filled=False).encode(
#     x = alt.X('min'),
#     x2 = alt.X2('max')
# )
rect_chart = alt.Chart(vis_df).mark_rect(opacity = 0.2).encode(
    x = alt.X('min'),
    x2 = alt.X2('max')
)
big_chart(ts_chart +rect_chart)

In [ ]:
chart = alt.Chart(profile_vis[(profile_vis.time.dt.isocalendar().week == 17)], width = 800, height = 200).mark_line(clip = True).encode(
    x = alt.X('time:T', title = None),
    y = alt.Y('value:Q', title = 'Consumption', scale = alt.Scale(domain = [0,1.0]))
)
big_chart(chart)

In [ ]:
IDX1, IDX2 = 8,22
def day_chart(idx, maxvalue = 2):
    profile_vis = (
        data_df.iloc[idx]
        .to_frame("value")
        .rename_axis(index = 'timestamp')
        .assign(
            date = lambda x: x.index.date.astype('str'), 
            time = lambda x: x.index.to_series().apply(lambda y: y.replace(year = 2016, month = 1, day = 1))
        )
        .reset_index())

    chart = alt.Chart(profile_vis, height = 200).mark_line(strokeWidth = 0.2, opacity = 0.4, clip = True).encode(
        x = alt.X('time:T', axis = alt.Axis(format = "%H:%M")),
        y = alt.Y('value:Q', scale = alt.Scale(domain = [0,maxvalue]), title = 'Consumption'),
        color = alt.Color('date', scale = alt.Scale(scheme = 'rainbow'), legend = None)
    )
    return chart

big_chart(alt.hconcat(day_chart(IDX1).properties(title = 'Profile 1'), day_chart(IDX2).properties(title = 'Profile 2')).resolve_scale(y = 'shared'), fontsize = 16)

In [ ]:
day_chart(IDX1)

In [ ]:
        
def add_date(series): 
    return pd.to_datetime(series, format='%H:%M:%S', exact = False)

def energy_heatmap_chart(meterID, data_df): 
    subset = (
        data_df.iloc[[meterID],:]
#         .droplevel(level = 1, axis =0)
        .stack().to_frame('value')
        .reset_index()
        .assign(
            time = lambda x: add_date(x.timestamp.dt.time),
            date = lambda x: x.timestamp.dt.date.astype('str'),
        )
    )
    return alt.Chart(subset, height = 1000, width = 1000).mark_rect(strokeOpacity = 0).encode(
        x = alt.X('time:O', axis = alt.Axis(labels = False, grid = False)),
        y = alt.Y('date:O', axis = alt.Axis(labels = False, grid = False)), 
        color = alt.Color('value:Q', scale = alt.Scale(scheme = 'viridis'))
    )
energy_heatmap_chart(IDX1, data_df)

In [ ]:
IDX = 22
profile_vis = (
        data_df.iloc[IDX]
        .to_frame("value")
        .rename_axis(index = 'timestamp')
        .assign(
            date = lambda x: x.index.date.astype('str'), 
            time = lambda x: x.index.to_series().apply(lambda y: y.replace(year = 2016, month = 1, day = 1))
        )
        .pipe(lambda x: pd.pivot_table(x, index = 'date', columns = 'time', values = 'value'))
        .dropna(axis = 0)
)
decomposer = NMF(10, max_iter = 100000, alpha = 0, l1_ratio = 0.9,  init = 'nndsvd').fit(profile_vis.to_numpy())
print('reconstruction error', decomposer.reconstruction_err_)
components = decomposer.components_
components_df = pd.DataFrame(components, columns = profile_vis.columns)
components_df.index.name = 'component_nb'
components_df

In [ ]:
from component_matching import get_scaled_components

In [ ]:
scaled_components, times_used = get_scaled_components(data_df.iloc[IDX])
scaled_components.columns = components_df.columns

In [ ]:
component_vis = scaled_components.stack().to_frame('value').reset_index()
component_vis = component_vis[component_vis.level_0.isin(range(0,5))]

In [ ]:
component_chart = alt.Chart(component_vis, height = 200).mark_line().encode(
    x = alt.X('time:T', axis = alt.Axis(format = '%H:%m')), 
    y = 'value:Q', 
    color= alt.Color('level_0:N', title = 'Pattern')
)
big_chart(( day_chart(IDX, maxvalue = 1).properties(title = 'Profile') | component_chart.properties(title = 'NMF components')).resolve_scale(color = 'independent', x = 'shared'), fontsize = 16)